<a href="https://colab.research.google.com/github/nvrancovich/futbol/blob/main/cotizaciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
import pandas as pd
import requests

In [37]:
pd.set_option('display.max_rows', None)

In [44]:
urls = {
    'https://www.transfermarkt.co/liga-profesional-de-futbol/startseite/wettbewerb/AR1N/plus/?saison_id=2016':'2016-2017',
    'https://www.transfermarkt.co/liga-profesional-de-futbol/startseite/wettbewerb/AR1N/plus/?saison_id=2017':'2017-2018',
    'https://www.transfermarkt.co/liga-profesional-de-futbol/startseite/wettbewerb/AR1N/plus/?saison_id=2018':'2018-2019',
    'https://www.transfermarkt.co/liga-profesional-de-futbol/startseite/wettbewerb/AR1N/plus/?saison_id=2019':'2019-2020',
    'https://www.transfermarkt.co/liga-profesional-de-futbol/startseite/wettbewerb/AR1N/plus/?saison_id=2020':'2021',
    'https://www.transfermarkt.co/liga-profesional-de-futbol/startseite/wettbewerb/AR1N/plus/?saison_id=2021':'2022'
}

In [22]:
headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

In [45]:
temporadas = []

for url in urls:
  req = requests.get(url, headers=headers)

  temporada = urls[url]
  print(temporada)

  df = pd.read_html(req.content.decode("utf-8"))[1]

  def formato_valor(valor):
    if ' mill. €' in valor:
      valor = int(float(valor.replace(' mill. €','').replace(',','.'))*1000000)
    elif ' mil €' in valor:
      valor = int(float(valor.replace(' mil €','').replace(',','.'))*100000)
    return valor

  df['temporada'] = temporada
  df = df[['temporada','Club.1','ø-Valor de mercado']][:-1]
  df = df.rename(columns={'Club.1':'club','ø-Valor de mercado':'valor_mercado_€'})
  df['valor_mercado_€'] = df['valor_mercado_€'].apply(formato_valor)

  clubes = {
      'Sarmiento':'CA Sarmiento (Junín)', 
      'Gimnasia ELP':'Club de Gimnasia y Esgrima La Plata', 
      'Rosario Cent':'CA Rosario Central', 
      'Racing Club':'Racing Club',
      'Tucumán':'Club Atlético Tucumán', 
      'San Lorenzo':'CA San Lorenzo de Almagro', 
      'Godoy Cruz':'CD Godoy Cruz Antonio Tomba', 
      'Tigre':'Club Atlético Tigre', 
      'Belgrano':'Club Atlético Belgrano', 
      'CA Unión':'Club Atlético Unión', 
      'River Plate':'CA River Plate', 
      'Lanús':'Club Atlético Lanús', 
      'Aldosivi':'Club Atlético Aldosivi',
      'Patronato':'Club Atlético Patronato', 
      'Banfield':'CA Banfield',
      'Defensa y Just':'CSD Defensa y Justicia', 
      'Independiente':'CA Independiente', 
      'CA Huracán':'CA Huracán',
      "Newell's OB":"CA Newell's Old Boys", 
      'Vélez Sarsf':'CA Vélez Sarsfield', 
      'Estudiantes':'Club Estudiantes de La Plata', 
      'Boca Juniors':'CA Boca Juniors',
      'Talleres':'Club Atlético Talleres', 
      'Arsenal':'Arsenal Fútbol Club', 
      'Colón':'Club Atlético Colón', 
      'Argentinos Jun':'AA Argentinos Juniors', 
      'CC Córdoba':'CA Central Córdoba (SdE)', 
      'Platense':'CA Platense',
      'Barracas Central':'CA Barracas Central',
      'Temperley':'Club Atlético Temperley', 
      'Club Olimpo':'Club Olimpo', 
      'San Martín SJ':'CA San Martín (San Juan)', 
      'Rafaela':'Atlético de Rafaela', 
      'Chacarita':'CA Chacarita Juniors',
      'San Martin':'CA San Martín (Tucumán)', 
      'Quilmes':'Quilmes Atlético Club'
  }   

  def nombres_clubes(club):
    for value in clubes:
      if club == clubes[value]:
        return value

  df['club'] = df['club'].apply(nombres_clubes)

  temporadas.append(df)

2016-2017
2017-2018
2018-2019
2019-2020
2021
2022


In [46]:
df = pd.concat(temporadas)

In [47]:
df.to_csv('/content/gdrive/MyDrive/proyectos/futbol/cotizaciones.csv', index=False)